In [ ]:
import numpy as np
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation

np.random.seed(42)

original_image = np.array([
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0]
])

print("Imaginea originala:")
print(original_image)

noisy_image = original_image.copy()
noisy_indices = np.random.choice(25, size=2, replace=False)

for index in noisy_indices:
    i = index // 5
    j = index % 5
    noisy_image[i, j] = 1 - noisy_image[i, j]

print("Imaginea cu zgomot:")
print(noisy_image)

lambda_param = 2.0
height = 5
width = 5
variables = []
for i in range(height):
    for j in range(width):
        variables.append(f"A_{i}_{j}")

edges = []
for i in range(height):
    for j in range(width):
        current = f"A_{i}_{j}"
        if j < width - 1:
            neighbor = f"A_{i}_{j+1}"
            edges.append((current, neighbor))
        if i < height - 1:
            neighbor = f"A_{i+1}_{j}"
            edges.append((current, neighbor))

model = MarkovNetwork()
model.add_nodes_from(variables)
model.add_edges_from(edges)

factors = []
for i in range(height):
    for j in range(width):
        var_name = f"A_{i}_{j}"
        observed_value = noisy_image[i, j]
        
        values_0 = np.exp(-lambda_param * (0 - observed_value)**2)
        values_1 = np.exp(-lambda_param * (1 - observed_value)**2)
        
        factor = DiscreteFactor(variables=[var_name],cardinality=[2],values=[values_0, values_1])
        factors.append(factor)

for edge in edges:
    var1, var2 = edge
    values = np.array([1.0, np.exp(-1), np.exp(-1), 1.0])
    factor = DiscreteFactor(variables=[var1, var2],cardinality=[2, 2],values=values.flatten())
    factors.append(factor)

model.add_factors(*factors)

bp = BeliefPropagation(model)
map_result = bp.map_query(variables=variables, show_progress=False)

denoised_image = np.array([[0]*5 for _ in range(5)])
for i in range(height):
    for j in range(width):
        var_name = f"A_{i}_{j}"
        denoised_image[i, j] = map_result[var_name]

print("Imaginea denoised:")
print(denoised_image)

accuracy_original = np.sum(original_image == noisy_image) / 25
accuracy_denoised = np.sum(original_image == denoised_image) / 25

print(f"Acuratetea imaginii cu zgomot: {accuracy_original:.2f}")
print(f"Acuratetea imaginii denoised: {accuracy_denoised:.2f}")

Imaginea originala:
[[0 0 1 1 1]
 [0 0 1 1 1]
 [0 0 0 1 1]
 [1 1 0 0 0]
 [1 1 0 0 0]]
Imaginea cu zgomot:
[[0 0 1 1 1]
 [0 0 1 0 1]
 [0 0 0 1 1]
 [1 0 0 0 0]
 [1 1 0 0 0]]
Imaginea denoised:
[[0 0 1 1 1]
 [0 0 1 1 1]
 [0 0 0 1 1]
 [1 0 0 0 0]
 [1 1 0 0 0]]
Acuratetea imaginii cu zgomot: 0.92
Acuratetea imaginii denoised: 0.96
